In [1]:
import requests as r
import random

In [2]:
def gen_event():
    event = {'lat': 70 + random.random()*5,
             'lon': 50 + random.random()*8,
             'ts': 1300000 + random.randint(0, 1000),
             'event': 0}
    return event

In [3]:
url = 'http://0.0.0.0:5000/'

In [23]:
method = 'processEvents'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

[{'cid': -1,
  'event': 0,
  'lat': 71.80574624437804,
  'lone': 51.75064239754856,
  'ts': 1300804},
 {'cid': -1,
  'event': 0,
  'lat': 74.71685858924404,
  'lone': 51.711011032961345,
  'ts': 1300839},
 {'cid': -1,
  'event': 0,
  'lat': 73.95593716997293,
  'lone': 51.369525385002056,
  'ts': 1300765}]

In [24]:
method = 'isClusterEmergency'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

{'prob': 1}

In [25]:
method = 'getClusterEmergencyReasons'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

{'reasons': {'unusual_high_activity': {'importance': 3.0, 'perc': 3.0}}}

# Develop check emergency

In [ ]:
import pandas as pd
import time
to_timestamp = lambda strdate: int(time.mktime(time.strptime(str(strdate), '%Y-%m-%d %H:%M:%S')))
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from models.cdbscan import Event, ContinuosCluster
from models.models import ClusteringModel

In [ ]:
df = pd.read_excel('../data/raw/Датасет3_Дтп.xlsx')
df['timestamp'] = list(map(to_timestamp, df['Время регистрации']))
df['lat'] = df['Широта']
df['lon'] = df['Долгота']
df = df.drop('Время регистрации	Категория	Идентификатор Еас адреса	Идентификатор Еас здания	Широта	Долгота	Район'.split('\t'), axis=1)

In [ ]:
def build_events(events_data):
    events = []
    for (ts, lat, lon) in events_data:
        events.append(Event(lat, lon, ts))
    return events

In [ ]:
last_ts = 1576200000
subdf = df[df.timestamp < last_ts]
subdf.shape, df.shape

In [ ]:
interval = 60*60*4 # by two hours

In [ ]:
start_ts = int(subdf.loc[0].timestamp.item())
stop_ts = int(subdf.loc[len(subdf)-1].timestamp.item())
model = ClusteringModel(0.01, 4, 60*10)

In [ ]:
# explore avg cluster info
avg_clusters_timestamps = []
avg_clusters = []

for start in tqdm(range(start_ts, stop_ts, interval//2)):
    events = subdf[(subdf.timestamp > start) & (subdf.timestamp < start+interval)]
    events = build_events(events.to_records(index=False))
    
    if len(events) < 2:
        continue
    
    labels = model.fit(events)
    clusters = [ContinuosCluster([]) for _ in range(max(0, max(labels)+1))]
    for i, event in enumerate(events):
        if labels[i] != -1:
            clusters[labels[i]].add(event)
    
    avg_clusters.append(clusters)
    avg_clusters_timestamps.append(start_ts + interval//2)

In [ ]:
# clusters count
_ = plt.figure(figsize=(20, 4))
_ = plt.plot(list(map(len, avg_clusters)))

In [ ]:
# avg points in clusters
_ = plt.figure(figsize=(20, 4))
def get_avg_points_n(avg_clusters):
    mean = lambda arr: sum(arr)/max(1, len(arr))
    avg_points_n = []
    for clusters in avg_clusters:
        avg_p_n = mean(list(map(lambda cl: len(cl.events), clusters)))
        avg_points_n.append(avg_p_n)
    return avg_points_n
        
_ = plt.plot(get_avg_points_n(avg_clusters))